## accesso al daily firstrate

In [19]:
from revelation.data.loading import (
    Catalog,
    firstrate_dirname,
    firstrate_filename,
    CSVPresets,
)
import pandas as pd

catalog = Catalog()

ohlc_d: pd.DataFrame = catalog.get_csv(
    catalog._raw_directory
    / "csv/firstrate"
    / firstrate_dirname()
    / firstrate_filename(),
    preset=CSVPresets.FIRSTRATE,
)

In [17]:
# print(ohlc_d.info())
print("is_unique", ohlc_d.index.is_unique)
print("is_sorted", ohlc_d.index.is_monotonic_increasing)

is_unique True
is_sorted True


In [ ]:
from revelation.visualization import set_chart

chart = set_chart(ohlc_d)
chart.load()

## accesso ai dati tradingview

In [20]:
import vectorbtpro as vbt

data = vbt.TVData.pull("NASDAQ:AAPL", timeframe="1 hour")

In [ ]:
data